In [3]:
%load_ext autoreload
%autoreload 2
import torch
import numpy as np
import pandas as pd
from omegaconf import OmegaConf
from pathlib import Path

# mycode
from ml_utilities.output_loader.result_loader import SweepResult, JobResult
from tflearning.mode_connectivity.linear_interpolation import interpolate_linear_runs
from ml_utilities.torch_utils.metrics import TError
from ml_utilities.data.datasetgenerator import DatasetGenerator

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
run_0 = JobResult('/system/user/beck/pwbeck/projects/regularization/tflearning/outputs/IA-A-cifar10-17.2.1-resnet-B--230120_134950')
run_0.get_summary()

,best_train_step,best_val_score,seed
IA-A-cifar10-17.2.1-resnet-B--230120_134950,44500,0.9166,0


In [5]:
run_0.get_data_log('val')['Accuracy'].argmax()

89

In [6]:
run_0.get_data_log('val').iloc[89,:]

log_step      89449.000000
loss              0.371612
Accuracy          0.916600
epoch           127.000000
train_step    44500.000000
Name: 89, dtype: float64

In [7]:
# display(run_0.get_data_log('val').style)

In [8]:
run_1 = JobResult('/system/user/beck/pwbeck/projects/regularization/tflearning/outputs/IA-B-cifar10-17.2.1-resnet-B--230120_141533/outputs/IA-B-cifar10-17.2.1-resnet-B--checkpoint_idx-0-seed-1--230120_161833')
run_1.get_summary()

,best_train_step,best_val_score,trainer.resume_training.checkpoint_idx,seed
IA-B-cifar10-17.2.1-resnet-B--checkpoint_idx-0-seed-1--230120_161833,64000,0.9134,0,1


In [21]:
data_cfg = """
data:
  dataset: cifar10
  dataset_kwargs:
    data_root_path: /system/user/beck/pwbeck/data
  dataset_split:
    train_val_split: 0.9
    restrict_n_samples_train_task: 10000
  # train_split_transforms:
    # image_transforms:
    # - RandomHorizontalFlip
    # - RandomCrop:
    #     size: 32
    #     padding: 4
    # tensor_transforms: 
    # joint_tensor_transforms: 
    # enable_transforms: True
"""
data_cfg = OmegaConf.create(data_cfg)

In [22]:
datagen = DatasetGenerator(**data_cfg.data)
datagen.generate_dataset()

Files already downloaded and verified


In [23]:
# ret = interpolate_linear_runs(run_0, run_1, score_fn=TError(), model_idx=[-2])

In [25]:
ret = interpolate_linear_runs(run_0, run_1, score_fn=TError(), interpolation_factors=[0.0, 1.0],model_idx=[16000,-1, -2], dataset_generator=datagen)

Alphas:  50%|█████     | 1/2 [01:30<01:30, 90.43s/it]


KeyboardInterrupt: 

In [14]:
ret[0]

datasets                                                              val  \
score                                                interpolation_scores   
alpha                                                                 0.0   
job                            seeds  model_idxes                           
IA-A-cifar10-17.2.1-resnet-B-- (0, 1) (16000, 16000)             0.180009   
                                      (44500, 64000)             0.136282   
                                      (64001, 64001)             0.136282   

datasets                                                                    \
score                                                          instability   
alpha                                                      1.0         NaN   
job                            seeds  model_idxes                            
IA-A-cifar10-17.2.1-resnet-B-- (0, 1) (16000, 16000)  0.161213    0.009398   
                                      (44500, 64000)  0.131273    0.002505   
                                      (64001, 64001)  0.131273    0.002505   

datasets                                                            train  \
score                                                interpolation_scores   
alpha                                                                 0.0   
job                            seeds  model_idxes                           
IA-A-cifar10-17.2.1-resnet-B-- (0, 1) (16000, 16000)                 0.17   
                                      (44500, 64000)                 0.05   
                                      (64001, 64001)                 0.05   

datasets                                                                
score                                                      instability  
alpha                                                  1.0         NaN  
job                            seeds  model_idxes                       
IA-A-cifar10-17.2.1-resnet-B-- (0, 1) (16000, 16000)  0.12       0.025  
                                      (44500, 64000)  0.05       0.000  
                                      (64001, 64001)  0.05       0.000

In [19]:
run_0.highest_model_idx, run_0.best_model_idx

(64001, 44500)

In [18]:
run_1.highest_model_idx, run_1.best_model_idx

(64001, 64000)